In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip install "peft==0.2.0"
# !pip install "transformers==4.27.2" "datasets==2.9.0" "accelerate==0.17.1" "evaluate==0.4.0" "bitsandbytes==0.37.1" loralib --upgrade --quiet

#pip install peft transformers datasets accelerate evaluate bitsandbytes loralib --upgrade --quiet
# install additional dependencies needed for training
# !pip install rouge-score tensorboard py7zr

In [ ]:
!pip install peft transformers datasets accelerate evaluate bitsandbytes loralib --upgrade --quiet

In [ ]:
!pip install --upgrade torch torchvision torchaudio

In [ ]:
!pip install tensorboard

In [ ]:
from datasets import load_from_disk, concatenate_datasets, DatasetDict

#textile patent documents
dataset = DatasetDict({
    'train': load_from_disk('/content/drive/MyDrive/FYP-Abstractive Text Summarization/Data/Textile_Patent_(70-20-10)_LexRank/train'),
    'validation': load_from_disk('/content/drive/MyDrive/FYP-Abstractive Text Summarization/Data/Textile_Patent_(70-20-10)_LexRank/validation'),
    'test': load_from_disk('/content/drive/MyDrive/FYP-Abstractive Text Summarization/Data/Textile_Patent_(70-20-10)_LexRank/test')
})
#dataset = load_from_disk('/content/drive/MyDrive/FYP-Abstractive Text Summarization/Data/Textile_Patent_(70-20-10)_LexRank')

In [ ]:
print(f"Train dataset size: {len(dataset['train'])}")
print(f"Validation dataset size: {len(dataset['validation'])}")
print(f"Test dataset size: {len(dataset['test'])}")

Train dataset size: 7905
Validation dataset size: 1130
Test dataset size: 2259


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="facebook/bart-base"

# Load tokenizer of FLAN-t5-SMALL
tokenizer = AutoTokenizer.from_pretrained(model_id)
#model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
#len(dataset["train"]['description'][4])
# min_len = 9999999
# for i in range(len(dataset['train'])):
#   temp = len(dataset["train"]['description'][i])
#   if (temp < min_len):
#     min_len = temp

# print(min_len)

In [ ]:
# from datasets import concatenate_datasets
# import numpy as np
# # The maximum total input sequence length after tokenization.
# # Sequences longer than this will be truncated, sequences shorter will be padded.
# tokenized_inputs = dataset["train"].map(lambda x: tokenizer(x["description"], truncation=True), batched=True, remove_columns=["description", "abstract"])
# input_lenghts = [len(x) for x in tokenized_inputs["input_ids"]]
# # take 85 percentile of max length for better utilization
# max_source_length = int(np.percentile(input_lenghts, 85))
# print(f"Max source length: {max_source_length}")

# # The maximum total sequence length for target text after tokenization.
# # Sequences longer than this will be truncated, sequences shorter will be padded."
# tokenized_targets = dataset["train"].map(lambda x: tokenizer(x["abstract"], truncation=True), batched=True, remove_columns=["description", "abstract"])
# target_lenghts = [len(x) for x in tokenized_targets["input_ids"]]
# # take 90 percentile of max length for better utilization
# max_target_length = int(np.percentile(target_lenghts, 90))
# print(f"Max target length: {max_target_length}")

In [ ]:
#Maximum available for bart model
max_source_length = 1024
max_target_length = 512

In [ ]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for bart
    #inputs = ["summarize: " + extract_important_sentences(item) for item in sample["description"]]
    inputs = ["summarize: " + item for item in sample["description"]]


    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["abstract"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["description", "abstract"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Map:   0%|          | 0/2259 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [ ]:
# save datasets to disk for later easy loading
tokenized_dataset["train"].save_to_disk("/content/drive/MyDrive/FYP-Abstractive Text Summarization/Adapter_Tuning/LoRA+Bart/tokenized_data/train")
tokenized_dataset["validation"].save_to_disk("/content/drive/MyDrive/FYP-Abstractive Text Summarization/Adapter_Tuning/LoRA+Bart/tokenized_data/validation")
tokenized_dataset["test"].save_to_disk("/content/drive/MyDrive/FYP-Abstractive Text Summarization/Adapter_Tuning/LoRA+Bart/tokenized_data/test")

Saving the dataset (0/1 shards):   0%|          | 0/7905 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1130 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2259 [00:00<?, ? examples/s]

In [ ]:
# # Load datasets
# tokenized_dataset = DatasetDict({
#     'train': load_from_disk('/content/drive/MyDrive/FYP-Abstractive Text Summarization/Adapter_Tuning/LoRA+Bart/tokenized_data/train'),
#     'validation': load_from_disk('/content/drive/MyDrive/FYP-Abstractive Text Summarization/Adapter_Tuning/LoRA+Bart/tokenized_data/validation'),
#     'test': load_from_disk('/content/drive/MyDrive/FYP-Abstractive Text Summarization/Adapter_Tuning/LoRA+Bart/tokenized_data/test')
# })

In [ ]:
# tokenized_data = {
#     'train': train,
#     'validation': valid,
#     'test': test
# }

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 7905
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1130
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2259
    })
})

In [ ]:
from transformers import AutoModelForSeq2SeqLM

# huggingface hub model id
model_id = "facebook/bart-base"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

In [ ]:
# !pip list | grep cuda

In [ ]:
print(model)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), 

In [ ]:
# import torch
# #check for gpu
# if torch.backends.mps.is_available():
#    mps_device = torch.device("mps")
#    x = torch.ones(1, device=mps_device)
#    print (x)
# else:
#    print ("MPS device not found.")

In [ ]:
import platform
print(platform.platform())

Linux-6.1.58+-x86_64-with-glibc2.35


In [ ]:
from peft import LoraConfig
from peft import get_peft_model
from peft import prepare_model_for_kbit_training
from peft import TaskType
# Define LoRA Config
lora_config = LoraConfig(
 r=16,
 lora_alpha=32,
 target_modules=["q_proj", "v_proj"],
 lora_dropout=0.05,
 bias="none",
 task_type=TaskType.SEQ_2_SEQ_LM
)
# prepare int-8 model for training
model = prepare_model_for_kbit_training(model)

# add LoRA adaptor
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 884,736 || all params: 140,305,152 || trainable%: 0.6305798378665383


In [ ]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

output_dir="lora-bart-base"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
	auto_find_batch_size=True,
    learning_rate=1e-3, # higher learning rate
    num_train_epochs=50,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=500,
    save_strategy="no",
    report_to="tensorboard",
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],

)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!


In [ ]:
# device = torch.device("mps:0")
# model.to(device)

In [ ]:
print(model)

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): BartForConditionalGeneration(
      (model): BartModel(
        (shared): Embedding(50265, 768, padding_idx=1)
        (encoder): BartEncoder(
          (embed_tokens): Embedding(50265, 768, padding_idx=1)
          (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
          (layers): ModuleList(
            (0-5): 6 x BartEncoderLayer(
              (self_attn): BartSdpaAttention(
                (k_proj): Linear(in_features=768, out_features=768, bias=True)
                (v_proj): lora.Linear(
                  (base_layer): Linear(in_features=768, out_features=768, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=768, out_features=16, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (defau

In [ ]:
# train model
trainer.train()

Step,Training Loss
500,3.099100
1000,2.931700
1500,2.855100
2000,2.823900
2500,2.797700
3000,2.762500
3500,2.725000
4000,2.726000
4500,2.703400
5000,2.708400


TrainOutput(global_step=49450, training_loss=2.5155551668944325, metrics={'train_runtime': 22427.128, 'train_samples_per_second': 17.624, 'train_steps_per_second': 2.205, 'total_flos': 2.43146842177536e+17, 'train_loss': 2.5155551668944325, 'epoch': 50.0})

In [ ]:
# Save our LoRA model & tokenizer results
peft_model_id="lora_textile_patent_Split_2_Exp_2"
trainer.model.save_pretrained(f'/content/drive/MyDrive/FYP-Abstractive Text Summarization/Adapter_Tuning/LoRA+Bart/{peft_model_id}_model')
tokenizer.save_pretrained(f'/content/drive/MyDrive/FYP-Abstractive Text Summarization/Adapter_Tuning/LoRA+Bart/{peft_model_id}_tokenizer')
# if you want to save the base model to call
# trainer.model.base_model.save_pretrained(peft_model_id)

('/content/drive/MyDrive/FYP-Abstractive Text Summarization/Adapter_Tuning/LoRA+Bart/lora_textile_patent_Split_2_Exp_2_tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/FYP-Abstractive Text Summarization/Adapter_Tuning/LoRA+Bart/lora_textile_patent_Split_2_Exp_2_tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/FYP-Abstractive Text Summarization/Adapter_Tuning/LoRA+Bart/lora_textile_patent_Split_2_Exp_2_tokenizer/vocab.json',
 '/content/drive/MyDrive/FYP-Abstractive Text Summarization/Adapter_Tuning/LoRA+Bart/lora_textile_patent_Split_2_Exp_2_tokenizer/merges.txt',
 '/content/drive/MyDrive/FYP-Abstractive Text Summarization/Adapter_Tuning/LoRA+Bart/lora_textile_patent_Split_2_Exp_2_tokenizer/added_tokens.json',
 '/content/drive/MyDrive/FYP-Abstractive Text Summarization/Adapter_Tuning/LoRA+Bart/lora_textile_patent_Split_2_Exp_2_tokenizer/tokenizer.json')

In [ ]:
test_dataset = load_from_disk("/content/drive/MyDrive/FYP-Abstractive Text Summarization/Adapter_Tuning/LoRA+T5/tokenized_data/test").with_format("torch")


In [ ]:
for sample in test_dataset:
 input = tokenizer.decode(sample['input_ids'].detach().cpu().numpy(), skip_special_tokens=True)
 output = model.generate(input_ids=sample["input_ids"].unsqueeze(0).cuda(), do_sample=True, top_p=0.9, max_new_tokens=max_target_length)
 prediction = tokenizer.decode(output[0].detach().cpu().numpy(), skip_special_tokens=True)
 break

In [ ]:
input

'summarize: CROSS-REFERENCE TO RELATED APPLICATIONS The present application claims priority from Australian Provisional Patent Application No 2006903417 filed on 26 Jun. 2006, the content of which is incorporated herein by reference. INTRODUCTION TO THE INVENTION This invention relates to the packaging of consumer products and relates particularly to products that are formed of pulp material, particularly pulp material formed of waste paper, waste timber, waste fabric material, virgin timber and other similar pulp material. The invention more particularly relates to high quality products carrying high definition printing or other decoration. BACKGROUND TO THE INVENTION The creation of compelling and high quality packaging for consumer durables is well established and is executed in a variety of forms and formats known in the prior art but with each of the prior art, the formats and methodologies having their own particular limitations. The consumer market demands increasing colour, vib

In [ ]:
prediction

'The present invention relates to product packaging which comprises high quality products carrying high definition printing and/or other similar products in the manufacture and/or any product that is able to be recycled.'